# First approach One hot encoding (Bolean encoding)
Analysis of an impact of a presence/absence activities on query of the another/other activity/ies presence/absence

In [15]:
import pandas as pd
import pm4py
import logview
from scipy.stats import chi2_contingency

Filtered log includes only:
- Start activities such as Create Purchase Order Item, Create Purchase Requisition Item, SRM Created, Vendor creates debit memo
- End activities Clear Invoice, Delete Purchase Order Item
- Only cases between 01.01.2018 and 31.12.2018


In [ ]:
log2019_filtered = pm4py.read_xes(r"")

parsing log, completed traces ::   0%|          | 0/169380 [00:00<?, ?it/s]

In [4]:
log19df_filtered= pm4py.convert_to_dataframe(log2019_filtered)
log19df_filtered = log19df_filtered.sort_values(['case:concept:name', 'time:timestamp'], ignore_index=True)
log2019_filtered = pm4py.format_dataframe(log19df_filtered, case_id='case:concept:name', activity_key='concept:name', timestamp_key='time:timestamp')

In [5]:
from logview.utils import LogViewBuilder

log_view_filtered = LogViewBuilder.build_log_view(log2019_filtered)

Hypothesis presence of activity Remove payment block

In [6]:
from logview.predicate import *

query_pb = Query('Remove Payment Block', [EqToConstant('concept:name', 'Remove Payment Block')])
result_set_query_pb, complement_pb = log_view_filtered.evaluate_query('rs_Remove payment block', log2019_filtered, query_pb)

result_set_query_pb.head(2)

,lifecycle:transition,time:timestamp,concept:name,org:resource,User,Cumulative net worth (EUR),case:concept:name,case:variant,case:variant-index,case:creator,...,case:Item Type,case:Item Category,case:Spend classification text,case:Source,case:Name,case:GR-Based Inv. Verif.,case:Item,case:Goods Receipt,@@index,@@case_index
76,complete,2018-01-08 13:12:00+00:00,SRM: Created,batch_00,batch_00,3227.0,2000000007_00001,Variant 169,169,Fluxicon Disco,...,Standard,"3-way match, invoice before GR",NPR,sourceSystemID_0000,vendor_0006,False,1,True,76,6
77,complete,2018-01-08 14:12:00+00:00,SRM: Complete,batch_00,batch_00,3227.0,2000000007_00001,Variant 169,169,Fluxicon Disco,...,Standard,"3-way match, invoice before GR",NPR,sourceSystemID_0000,vendor_0006,False,1,True,77,6


In [8]:
unique_events = sorted(log2019_filtered['concept:name'].unique())

In [9]:
result_set_query_pb_sorted = result_set_query_pb.sort_values(by=['case:concept:name', 'time:timestamp'])
complement_pb_sorted = complement_pb.sort_values(by=['case:concept:name', 'time:timestamp'])

result set encoding

In [14]:
result_set_query_pb_sorted_ohe = result_set_query_pb_sorted.drop_duplicates(subset=['case:concept:name', 'concept:name'])
encodedohe_result_set_query_pb = pd.crosstab(result_set_query_pb_sorted_ohe['case:concept:name'], result_set_query_pb_sorted_ohe['concept:name']).reindex(unique_events, axis=1, fill_value=0)

complement encoding

In [13]:
complement_pb_sorted_ohe = complement_pb_sorted.drop_duplicates(subset=['case:concept:name', 'concept:name'])
encodedohe_complement_pb = pd.crosstab(complement_pb_sorted_ohe['case:concept:name'], complement_pb_sorted_ohe['concept:name']).reindex(unique_events, axis=1, fill_value=0)

labeling

In [17]:
encodedohe_result_set_query_pb_labeled = encodedohe_result_set_query_pb.copy()
encodedohe_result_set_query_pb_labeled['label'] = 'with_remove'

encodedohe_complement_pb_labeled=encodedohe_complement_pb.copy()
encodedohe_complement_pb_labeled['label'] = 'without_remove'

df_combined = pd.concat([encodedohe_result_set_query_pb_labeled, encodedohe_complement_pb_labeled])

    

"Cancel Invoice Receipt", Record Invoice Receipt, and Set Payment Block, show significant differences, suggesting that they might be correlated with the occurrence of "Remove payment block."

In [23]:
def analyze_significant_relationships(df, significance_level=0.05):
    for column in df.columns[:-1]:  
        contingency_table = pd.crosstab(df[column], df['label'])
        chi2, p, dof, expected = chi2_contingency(contingency_table)
        print (contingency_table[0:5])
        print(f"Activity: {column}, Chi-Square p-value: {p:.2f}", end=", ")

        if p <= significance_level:
            observed_counts = contingency_table.values
            labels = contingency_table.columns

            if observed_counts.shape[0] > 1:
                counts = observed_counts[1]  
            else:
                counts = [0] * len(labels)

            max_count = max(counts)
            max_count_labels = [label for count, label in zip(counts, labels) if count == max_count]

            if len(max_count_labels) == 1:
                print(f"Significant Relationship with: {max_count_labels[0]}")
            else:
                print(f"Significant Relationship: Equal between {', '.join(max_count_labels)}")
        else:
            print("No significant relationship")

print("\nDetailed Analysis of Significant Relationships:")



Detailed Analysis of Significant Relationships:


In [24]:
analyze_significant_relationships(df_combined)

label                      with_remove  without_remove
Block Purchase Order Item                             
0                                43290          125843
1                                   63             184
Activity: Block Purchase Order Item, Chi-Square p-value: 1.00, No significant relationship
label                 with_remove  without_remove
Cancel Goods Receipt                             
0                           43098          125306
1                             255             721
Activity: Cancel Goods Receipt, Chi-Square p-value: 0.73, No significant relationship
label                   with_remove  without_remove
Cancel Invoice Receipt                             
0                             41174          124150
1                              2179            1877
Activity: Cancel Invoice Receipt, Chi-Square p-value: 0.00, Significant Relationship with: with_remove
label                      with_remove  without_remove
Cancel Subsequent Invoice            

In [ ]:
def analyze_activity_label_association(df, significance_level=0.05):
    for column in df.columns[:-1]:
        contingency_table = pd.crosstab(df[column], df['label'])
        chi2, p, dof, expected = chi2_contingency(contingency_table)

        print(f"=====================\nAnalyzing Activity: '{column}'")
        print(f"Chi-Square p-value: {p:.4f}", end=", ")

        if p <= significance_level:
            observed_counts = contingency_table.values
            labels = contingency_table.columns

            activity_0 = observed_counts[0]  
            activity_1 = observed_counts[1]  
            
            presence_label = labels[activity_1.argmax()] 
            absence_label = labels[activity_0.argmax()]  

            print(f"The presence of '{column}' activity is associated with the '{presence_label}' label.")
            print(f"The absence of '{column}' activity is associated with the '{absence_label}' label.")
        else:

            print(f"No significant relationship between the '{column}' activity and the labels.")

        print("=====================")

In [41]:
analyze_activity_label_association(df_combined)

Analyzing Activity: 'Block Purchase Order Item'
Chi-Square p-value: 1.0000, No significant relationship between the 'Block Purchase Order Item' activity and the labels.
Analyzing Activity: 'Cancel Goods Receipt'
Chi-Square p-value: 0.7300, No significant relationship between the 'Cancel Goods Receipt' activity and the labels.
Analyzing Activity: 'Cancel Invoice Receipt'
Chi-Square p-value: 0.0000, The presence of 'Cancel Invoice Receipt' activity is associated with the 'with_remove' label.
The absence of 'Cancel Invoice Receipt' activity is associated with the 'without_remove' label.
Analyzing Activity: 'Cancel Subsequent Invoice'
Chi-Square p-value: 0.0000, The presence of 'Cancel Subsequent Invoice' activity is associated with the 'with_remove' label.
The absence of 'Cancel Subsequent Invoice' activity is associated with the 'without_remove' label.
Analyzing Activity: 'Change Approval for Purchase Order'
Chi-Square p-value: 0.0022, The presence of 'Change Approval for Purchase Order'